In [ ]:
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_hub as hub
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [ ]:
print(f'Tensorflow 버전을 확인합니다: {tf.__version__}')

In [ ]:
(_train_images, train_labels), (_test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

In [ ]:
print(f'학습에 사용할 이미지는 {len(_train_images)}개 입니다.')
print(f'학습한 후 테스트(검증)에 사용할 이미지는 {len(_test_images)}개 입니다.')

In [ ]:
class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer',
               'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
print(f'데이터의 레이블은 {len(class_names)}개 이며, 데이터셋에 포함되어 있지 않으므로 설명서에서 확인해야 합니다.')
print('레이블 번호와 레이블: ')
for i in range(0, len(class_names)):
    print(f'{i}: {class_names[i]}')

In [ ]:
print('인공 신경망은 주로 -1.0 ~ 1.0 사이의 값을 받습니다.')
print('따라서 이미지를 255로 나누어줍니다.')
train_images = _train_images / 255.0
test_images = _test_images / 255.0
print(f'기존 최소: {np.min(_train_images[0])}, 기존 최대: {np.max(_train_images[0])}')
print(f'정규화 후 최소: {np.min(train_images[0])}, 정규화 후 최대: {np.max(train_images[0])}')

In [ ]:
fig = plt.figure(figsize=(12, 12))
fig.set_facecolor('white')
for i in range(25):
    ax = fig.add_subplot(5, 5, i+1)
    ax.imshow(train_images[i])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(class_names[train_labels[i][0]])

In [ ]:
fig = plt.figure(figsize=(12, 12/5*2))
fig.set_facecolor('white')
for i in range(10):
    ax = fig.add_subplot(2, 5, i+1)
    for j in range(10000, len(train_images)):
        if train_labels[j] == i:
            break
    ax.imshow(train_images[j])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f'[{train_labels[j][0]}] {class_names[train_labels[j][0]]}', fontsize=12)

In [ ]:
input_shape = train_images[0].shape
print(f'데이터 상태: {input_shape}')

In [ ]:
X = []
y = []
T = 2
for a, b in zip(train_images, train_labels):
    if T < b[0]:
        continue
    X.append(a)
    y.append(b)
X = np.asarray(X)
y = np.asarray(y)
print(f'{T} 이하 데이터 셋 크기: {X.shape}, {y.shape}')
input_shape = X[0].shape

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(T+1)
])
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model_init = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(T+1)
])
model_init.summary()

In [ ]:
model_init.layers[1].set_weights(model.layers[1].get_weights())

In [ ]:
epochs = 100
history = model.fit(X, y, 
                    epochs=epochs,
                    verbose=0)

In [ ]:
model.evaluate(X, y)

### Training

In [ ]:
X = []
y = []
T = 1

for a, b in zip(train_images, train_labels):
    if T < b[0]:
        continue
    X.append(a)
    y.append(b)
X = np.asarray(X)
y = np.asarray(y)
print(f'{T} 이하 데이터 셋 크기: {X.shape}, {y.shape}')
input_shape = X[0].shape

model_b1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(T+1)
])
model_b1.summary()
model_b1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

epochs = 100
history = model_b1.fit(X, y, 
                    epochs=epochs,
                    verbose=0)
model_b1.evaluate(X, y)

In [ ]:
X = []
y = []
T = 2

for a, b in zip(train_images, train_labels):
    if T < b[0]:
        continue
    X.append(a)
    y.append(b)
X = np.asarray(X)
y = np.asarray(y)
print(f'{T} 이하 데이터 셋 크기: {X.shape}, {y.shape}')
input_shape = X[0].shape

model1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(T+1)
])
model1.summary()
model1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model1.layers[1].set_weights([np.pad(model_b1.layers[1].get_weights()[0], (0, 1), mode='constant')[:-1, :], 
                              np.pad(model_b1.layers[1].get_weights()[1], (0, 1), mode='constant')])

In [ ]:
epochs = 100
history = model1.fit(X, y, 
                    epochs=epochs,
                    verbose=0)
model1.evaluate(X, y)

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(T+1)
])
model2.summary()
model2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model2.layers[1].set_weights([np.pad(model_b1.layers[1].get_weights()[0], (0, 1), mode='constant')[:-1, :], 
                              np.pad(model_b1.layers[1].get_weights()[1], (0, 1), mode='constant')])

In [ ]:
epochs = 100
history = model2.fit(X, y, 
                    epochs=epochs,
                    verbose=0)
model2.evaluate(X, y)

### Comparing

In [ ]:
# Need https://github.com/yuanli2333/CKA-Centered-Kernel-Alignment
import cca_core
from CKA import linear_CKA, kernel_CKA

In [ ]:
print('Base and Expand 1')
print('Linear CKA: {}'.format(linear_CKA(model_b1.layers[1].get_weights()[0], model1.layers[1].get_weights()[0])))
print('Kernel CKA: {}'.format(kernel_CKA(model_b1.layers[1].get_weights()[0], model1.layers[1].get_weights()[0])))

In [ ]:
print('Base and Expand 2')
print('Linear CKA: {}'.format(linear_CKA(model_b1.layers[1].get_weights()[0], model2.layers[1].get_weights()[0])))
print('Kernel CKA: {}'.format(kernel_CKA(model_b1.layers[1].get_weights()[0], model2.layers[1].get_weights()[0])))

In [ ]:
print('Base and Random')
print('Linear CKA: {}'.format(linear_CKA(model_b1.layers[1].get_weights()[0], model.layers[1].get_weights()[0])))
print('Kernel CKA: {}'.format(kernel_CKA(model_b1.layers[1].get_weights()[0], model.layers[1].get_weights()[0])))

In [ ]:
print('Expand 1 and Expand 2')
print('Linear CKA: {}'.format(linear_CKA(model1.layers[1].get_weights()[0], model2.layers[1].get_weights()[0])))
print('Kernel CKA: {}'.format(kernel_CKA(model1.layers[1].get_weights()[0], model2.layers[1].get_weights()[0])))

In [ ]:
print('Random and Expand 1')
print('Linear CKA: {}'.format(linear_CKA(model.layers[1].get_weights()[0], model1.layers[1].get_weights()[0])))
print('Kernel CKA: {}'.format(kernel_CKA(model.layers[1].get_weights()[0], model1.layers[1].get_weights()[0])))

In [ ]:
print('Random and Expand 2')
print('Linear CKA: {}'.format(linear_CKA(model.layers[1].get_weights()[0], model2.layers[1].get_weights()[0])))
print('Kernel CKA: {}'.format(kernel_CKA(model.layers[1].get_weights()[0], model2.layers[1].get_weights()[0])))

In [ ]:
model_b1.layers[1].get_weights()[1]

In [ ]:
model1.layers[1].get_weights()[1]

In [ ]:
model2.layers[1].get_weights()[1]

In [ ]:
model.layers[1].get_weights()[1]

In [ ]:
model.layers[1].get_weights()[0]

In [ ]:
model_b1.layers[1].get_weights()[0]

In [ ]:
model1.layers[1].get_weights()[0]

In [ ]:
model2.layers[1].get_weights()[0]